# Homework 1 (Total Points: 260) <a class="anchor" id="top"></a>


**Submission instructions**:
- Only the code `TODO: Implement this!` denotes that these sections are graded.
- For Part 1: You can use the `nltk`, `numpy` and `matplotlib` libraries here. Other libraries, e.g., `gensim` or `scikit-learn`, may not be used. For Part 2: `gensim` is allowed in addition to the imported libraries in the next code cell
- The notebook you submit has to have the student ids, separated by underscores (E.g., `12341234_12341234_12341234.ipynb`). 
- This will be parsed by a regexp, so please double check your filename.
- Only one member of each group has to submit the file to canvas.
- Make sure to check that your notebook runs before submission. A quick way to do this is to restart the kernel and run all the cells.  
- Please do not delete/add new cells. Removing cells can lead to grade deduction.
- Note, that you are not allowed to use Google Colab.


**Learning Goals**:
- [Part 1, Term-based matching](#part1) (175 points):
    - Learn how to load a dataset and process it.
    - Learn how to implement several standard IR methods (TF-IDF, BM25, QL) and understand their weaknesses & strengths.
    - Learn how to evaluate IR methods.
- [Part 2, Semantic-based matching](#part2) (85 points):
    - Learn how to implement vector-space retrieval methods (LSI, LDA).
    - Learn how to use LSI and LDA for re-ranking

    
**Resources**: 
- **Part 1**: Sections 2.3, 4.1, 4.2, 4.3, 5.3, 5.6, 5.7, 6.2, 7, 8 of [Search Engines: Information Retrieval in Practice](https://ciir.cs.umass.edu/downloads/SEIRiP.pdf)
- **Part 2**: [LSI - Chapter 18](https://nlp.stanford.edu/IR-book/pdf/18lsi.pdf) from [Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/) book and the [original LDA paper](https://jmlr.org/papers/volume3/blei03a/blei03a.pdf)

In [1]:
# imports 
# TODO: Ensure that no additional library is imported in the notebook. 
# TODO: Only the standard library and the following libraries are allowed:
# TODO: You can also use unlisted classes from these libraries or standard libraries (such as defaultdict, Counter, ...).

import os
import zipfile
from functools import partial

import nltk
import requests
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from ipywidgets import widgets
from IPython.display import display, HTML
#from IPython.html import widgets
from collections import namedtuple

# Own imports from standard libraries
from collections import Counter
import math



%matplotlib inline


# Part 1: Term-based Matching (175 points) <a class="anchor" id="part1"></a>

[Back to top](#top)

In the first part, we will learn the basics of IR from loading and preprocessing the material, to implementing some well known search algorithms, to evaluating the ranking performance of the implemented algorithms. We will be using the CACM dataset throughout the assignment. The CACM dataset is a collection of titles and abstracts from the journal CACM (Communication of the ACM).

Table of contents:
- [Section 1: Text Processing](#text_processing) (15 points)
- [Section 2: Indexing](#indexing) (10 points)
- [Section 3: Ranking](#ranking) (80 points)
- [Section 4: Evaluation](#evaluation) (40 points)
- [Section 5: Analysis](#analysis) (30 points)


---
## Section 1: Text Processing (15 points)<a class="anchor" id="text_processing"></a>

[Back to Part 1](#part1)

In this section, we will load the dataset and learn how to clean up the data to make it usable for an IR system. 
The points of this section are earned for the following implementations:
- `read_cacm_docs` (4 points): Reads in the CACM documents.
- `read_queries` (3 points): Reads in the CACM queries.
- `load_stopwords` (1 point): Loads the stopwords.
- `tokenize` (4 points): Tokenizes the input text.
- `stem_token` (3 points): Stems the given token. 

We are using the [CACM dataset](http://ir.dcs.gla.ac.uk/resources/test_collections/cacm/), which is a small, classic IR dataset, composed of a collection of titles and abstracts from the journal CACM. It comes with relevance judgements for queries, so we can evaluate our IR system. 


---
### 1.1 Read the CACM documents (4 points)


The following cell downloads the dataset and unzips it to a local directory.

In [2]:
def download_dataset():
    folder_path = os.environ.get("IR1_DATA_PATH")
    if not folder_path:
        folder_path = "./datasets/"
    os.makedirs(folder_path, exist_ok=True)
    
    file_location = os.path.join(folder_path, "cacm.zip")
    
    # download file if it doesn't exist
    if not os.path.exists(file_location):
        
        url = "https://surfdrive.surf.nl/files/index.php/s/M0FGJpX2p8wDwxR/download"

        with open(file_location, "wb") as handle:
            print(f"Downloading file from {url} to {file_location}")
            response = requests.get(url, stream=True)
            for data in tqdm(response.iter_content()):
                handle.write(data)
            print("Finished downloading file")
    
    if not os.path.exists(os.path.join(folder_path, "train.txt")):
        
        # unzip file
        with zipfile.ZipFile(file_location, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
        
download_dataset()

---

You can see a brief description of each file in the dataset by looking at the README file:

In [3]:
##### Read the README file 
!cat ./datasets/README
#####

Files in this directory with sizes:
          0 Jun 19 21:01 README

    2187734 Jun 19 20:55 cacm.all              text of documents
        626 Jun 19 20:58 cite.info             key to citation info
                                                (the X sections in cacm.all)
       2668 Jun 19 20:55 common_words           stop words used by smart
       2194 Jun 19 20:55 make_coll*             shell script to make collection
       1557 Jun 19 20:55 make_coll_term*        ditto (both useless without
                                                smart system)
       9948 Jun 19 20:55 qrels.text             relation giving
                                                    qid did 0 0
                                                to indicate dument did is
                                                relevant to query qid
      13689 Jun 19 20:55 query.text             Original text of the query


---
We are interested in 4 files:
- `cacm.all` : Contains the text for all documents. Note that some documents do not have abstracts available. 
- `query.text` : The text of all queries
- `qrels.text` : The relevance judgements
- `common_words` : A list of common words. This may be used as a collection of stopwords

In [4]:
##### The first 45 lines of the CACM dataset forms the first record
# We are interested only in 3 fields. 
# 1. the '.I' field, which is the document id
# 2. the '.T' field (the title) and
# 3. the '.W' field (the abstract, which may be absent)
!head -45 ./datasets/cacm.all
#####

.I 1
.T
Preliminary Report-International Algebraic Language
.B
CACM December, 1958
.A
Perlis, A. J.
Samelson,K.
.N
CA581203 JB March 22, 1978  8:28 PM
.X
100	5	1
123	5	1
164	5	1
1	5	1
1	5	1
1	5	1
205	5	1
210	5	1
214	5	1
1982	5	1
398	5	1
642	5	1
669	5	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
165	6	1
196	6	1
196	6	1
1273	6	1
1883	6	1
324	6	1
43	6	1
53	6	1
91	6	1
410	6	1
3184	6	1


---

**Implementation (4 points):**
Write a function to read the `cacm.all` file. Note that each document has a variable number of lines. The `.I` field denotes a new document:

In [5]:
# TODO: Implement this! (4 points)
def read_cacm_docs(root_folder = "./datasets/"):
    """
        Reads in the CACM documents. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (doc_id, document), where 'document' is a single string created by 
            appending the title and abstract (separated by a "\n"). 
            In case the record doesn't have an abstract, the document is composed only by the title
    """
    # YOUR CODE HERE
    doc_list = []
    with open(root_folder + 'cacm.all') as f:
        data = ("\n" + f.read()).split("\n.I")
        for i in data[1:]:
            lines = i.split("\n.")
            doc_id = str(lines[0][1:])
            document = ''
            for line in lines[1:]:
                if line[0] == "T":
                    document += line[2:]
                if line[0] == "W":
                    document += line[1:]
            doc_list.append((doc_id,document))
    return doc_list

In [6]:
##### Function check
docs = read_cacm_docs()
assert isinstance(docs, list)
assert len(docs) == 3204, "There should be exactly 3024 documents"
##### 

---
### 1.2 Read the CACM queries (3 points)

Next, let us read the queries. They are formatted similarly:

In [7]:
##### The first 15 lines of 'query.text' has 2 queries
# We are interested only in 2 fields. 
# 1. the '.I' - the query id
# 2. the '.W' - the query
!head -15 ./datasets/query.text
#####

.I 1
.W
 What articles exist which deal with TSS (Time Sharing System), an
operating system for IBM computers?
.N
 1. Richard Alexander, Comp Serv, Langmuir Lab (TSS)
 
.I 2
.W
 I am interested in articles written either by Prieve or Udo Pooch
.A
Prieve, B.
Pooch, U.
.N
 2. Richard Alexander, Comp Serv, Langmuir Lab (author = Pooch or Prieve)


---

**Implementation (3 points):**
Write a function to read the `query.text` file:

In [8]:
# TODO: Implement this! (3 points)
def read_queries(root_folder = "./datasets/"):
    """
        Reads in the CACM queries. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (query_id, query)
    """
    # YOUR CODE HERE
    query_list = []
    with open(root_folder + 'query.text') as f:
        data = ("\n" + f.read()).split("\n.I")
        for i in data[1:]:
            lines = i.split("\n.")
            query_id = str(lines[0][1:])
            query = ''
            for line in lines[1:]:
                if line[0] == "W":
                    query += line[1:]
            query_list.append((query_id,query))
    print(query_list)
    return query_list


In [9]:
##### Function check
queries = read_queries()

assert isinstance(queries, list)
assert len(queries) == 64 and all([q[1] is not None for q in queries]), "There should be exactly 64 queries"
##### 

[('1', '\n What articles exist which deal with TSS (Time Sharing System), an\noperating system for IBM computers?'), ('2', '\n I am interested in articles written either by Prieve or Udo Pooch'), ('3', '\n Intermediate languages used in construction of multi-targeted compilers; TCOLL'), ('4', "\n I'm interested in mechanisms for communicating between disjoint processes,\npossibly, but not exclusively, in a distributed environment.  I would\nrather see descriptions of complete mechanisms, with or without implementations,\nas opposed to theoretical work on the abstract problem.  Remote procedure\ncalls and message-passing are examples of my interests."), ('5', "\n I'd like papers on design and implementation of editing interfaces,\nwindow-managers, command interpreters, etc.  The essential issues are\nhuman interface design, with views on improvements to user efficiency,\neffectiveness and satisfaction."), ('6', '\n Interested in articles on robotics, motion planning particularly the\nge

---
### 1.3 Read the stop words (1 point)

We use the common words stored in `common_words`:

In [10]:
##### Read the stop words file 
!head ./datasets/common_words
##### Read the README file 

a
about
above
accordingly
across
after
afterwards
again
against
all


---
**Implementation (1 point):**
Write a function to read the `common_words` file (For better coverage, try to keep them in lowercase):

In [11]:
# TODO: Implement this! (1 point)
def load_stopwords(root_folder = "./datasets/"):
    """
        Loads the stopwords. The dataset is assumed to be in the folder "./datasets/" by default
        Output: A set of stopwords
    """
    # YOUR CODE HERE
    with open(os.path.join(root_folder, "common_words"), 'r+') as f:
        words = set(f.readlines())
        
    
    return words

In [12]:
##### Function check
stopwords = load_stopwords()

assert isinstance(stopwords, set)
assert len(stopwords) == 428, "There should be exactly 428 stop words"
##### 

---
### 1.4 Tokenization (4 points)

We can now write some basic text processing functions. 
A first step is to tokenize the text. 

**Note**: Use the  `WordPunctTokenizer` available in the `nltk` library:

In [13]:
# TODO: Implement this! (4 points)
def tokenize(text):
    """
        Tokenizes the input text. Use the WordPunctTokenizer
        Input: text - a string
        Output: a list of tokens
    """
    # YOUR CODE HERE
    return nltk.tokenize.WordPunctTokenizer().tokenize(text)
    

In [14]:
##### Function check
text = "the quick brown fox jumps over the lazy dog"
tokens = tokenize(text)

assert isinstance(tokens, list)
assert len(tokens) == 9

print(tokens)
# output: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
#####

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


---
### 1.5 Stemming (3 points)

Write a function to stem tokens. 
Again, you can use the nltk library for this:

In [15]:
# TODO: Implement this! (3 points)
def stem_token(token):
    """
        Stems the given token using the PorterStemmer from the nltk library
        Input: a single token
        Output: the stem of the token
    """
    ps = nltk.stem.PorterStemmer()
    return ps.stem(token)


In [16]:
##### Function check
assert stem_token('owned') == 'own'
assert stem_token('itemization') == 'item'
#####

---
### 1.6 Summary

The following function puts it all together. Given a string, it tokenizes it and processes it according to the flags that you set.

In [17]:
#### Putting it all together
def process_text(text, stem=False, remove_stopwords=False, lowercase_text=False):
    
    tokens = []
    for token in tokenize(text):
        if remove_stopwords and token.lower() in stopwords:
            continue
        if stem:
            token = stem_token(token)
        if lowercase_text:
            token = token.lower()
        tokens.append(token)

    return tokens
#### 

---

Let's create two sets of preprocessed documents.
We can process the documents and queries according to these two configurations:

In [18]:
# In this configuration:
# Don't preprocess the text, except to tokenize 
config_1 = {
  "stem": False,
  "remove_stopwords" : False,
  "lowercase_text": True
} 


# In this configuration:
# Preprocess the text, stem and remove stopwords
config_2 = {
  "stem": True,
  "remove_stopwords" : True,
  "lowercase_text": True, 
} 

####
doc_repr_1 = []
doc_repr_2 = []
for (doc_id, document) in docs:
    doc_repr_1.append((doc_id, process_text(document, **config_1)))
    doc_repr_2.append((doc_id, process_text(document, **config_2)))

####

--- 

## Section 2: Indexing (10 points)<a class="anchor" id="indexing"></a>

[Back to Part 1](#part1)



A retrieval function usually takes in a query document pair, and scores a query against a document.  Our document set is quite small - just a few thousand documents. However, consider a web-scale dataset with a few million documents. In such a scenario, it would become infeasible to score every query and document pair. In such a case, we can build an inverted index. From Wikipedia:

> ... , an inverted index (also referred to as a postings file or inverted file) is a database index storing a mapping from content, such as words or numbers, to its locations in a table, .... The purpose of an inverted index is to allow fast full-text searches, at a cost of increased processing when a document is added to the database. ...


Consider a simple inverted index, which maps from word to document. This can improve the performance of a retrieval system significantly. In this assignment, we consider a *simple* inverted index, which maps a word to a set of documents. In practice, however, more complex indices might be used.  


### 2.1 Term Frequency-index (10 points)
In this assignment, we will be using an index created in memory since our dataset is tiny. To get started, build a simple index that maps each `token` to a list of `(doc_id, count)` where `count` is the count of the `token` in `doc_id`.
For consistency, build this index using a python dictionary.
    
Now, implement a function to build an index:

In [19]:
# TODO: Implement this! (10 points)
def build_tf_index(documents):
    """
        Build an inverted index (with counts). The output is a dictionary which takes in a token
        and returns a list of (doc_id, count) where 'count' is the count of the 'token' in 'doc_id'
        Input: a list of documents - (doc_id, tokens) 
        Output: An inverted index. [token] -> [(doc_id, token_count)]
    """
    # YOUR CODE HERE
    inverted_index = {}
    for (doc_id, words) in documents:
        word_occurences = Counter(words)        
        for word in word_occurences:
            token_count = word_occurences[word]
            if word not in inverted_index:
               inverted_index[word] = [(doc_id,token_count)]
            else:
                inverted_index[word].append((doc_id, token_count))

    return inverted_index

---
Now we can build indexed documents and preprocess the queries based on the two configurations:

In [20]:
#### Indexed documents based on the two configs

# Create the 2 indices
tf_index_1 = build_tf_index(doc_repr_1)
tf_index_2 = build_tf_index(doc_repr_2)

# This function returns the tf_index of the corresponding config
def get_index(index_set):
    assert index_set in {1, 2}
    return {
        1: tf_index_1,
        2: tf_index_2
    }[index_set]

####
#### Preprocessed query based on the two configs

# This function preprocesses the text given the index set, according to the specified config
def preprocess_query(text, index_set):
    assert index_set in {1, 2}
    if index_set == 1:
        return process_text(text, **config_1)
    elif index_set == 2:
        return process_text(text, **config_2)

#### 

In [21]:
##### Function check

print(tf_index_1['computer'][:10])
#### 

[('4', 1), ('7', 1), ('10', 1), ('13', 1), ('19', 1), ('22', 1), ('23', 1), ('37', 1), ('40', 3), ('41', 1)]


In [22]:
##### Function check

print(tf_index_2['computer'][:10]) 
#### 

[('670', 1), ('675', 1), ('1621', 3), ('1681', 1), ('2145', 1), ('2339', 1), ('2572', 1), ('2583', 1), ('2739', 1), ('3012', 1)]



---
## Section 3: Ranking  (80 points) <a class="anchor" id="ranking"></a>

[Back to Part 1](#part1)

Now that we have cleaned and processed our dataset, we can start building simple IR systems. 

For now, we consider *simple* IR systems, which involve computing scores from the tokens present in the document/query. More advanced methods are covered in later assignments.

We will implement the following methods in this section:
- [Section 3.1: Bag of Words](#bow) (10 points)
- [Section 3.2: TF-IDF](#tfidf) (15 points)
- [Section 3.3: Query Likelihood Model](#qlm) (35 points)
- [Section 3.4: BM25](#bm25) (20 points)

**Scoring policy:**
Your implementations in this section are scored based on the expected performance of your ranking functions.
You will get a full mark if your implementation meets the expected performance (measured by some evaluation metric).
Otherwise, you may get partial credit.
For example, if your *Bag of words* ranking function has 60% of expected performance, you will get 4 out of 10.

--- 

### Section 3.1: Bag of Words (10 points)<a class="anchor" id="bow"></a>

Probably the simplest IR model is the Bag of Words (BOW) model.
Implement a function that scores and ranks all the documents against a query using this model.   

Note that you can use either the count of the token or 'binarize' it i.e set the value equal to 1 if the token appears.   


In [103]:
# TODO: Implement this! (10 points)
def bow_search(query, index_set):
    """
        Perform a search over all documents with the given query. 
        Note: You have to use the `get_index` function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    doc_scores = {}
    for query_word in processed_query:
        if query_word in index:
            word_occurences = index[query_word] 
            for (doc_id, counts) in word_occurences:
                if doc_id not in doc_scores:
                    doc_scores[doc_id] = 1
                else:
                    doc_scores[doc_id] += 1
    
    sorted_scores = [(k, float(v)) for k, v in sorted(doc_scores.items(), key=lambda item:item[1], reverse=True)]

    return sorted_scores

In [104]:
#### Function check

docs_by_id = dict(docs)
def print_results(docs, len_limit=50):    
    for i, (doc_id, score) in enumerate(docs):
        doc_content = docs_by_id[doc_id].strip().replace("\n", "\\n")[:len_limit] + "..."
        print(f"Rank {i}({score:.2}): {doc_content}")

test_bow = bow_search("report", index_set=1)[:5]
print(f"BOW Results:")
print_results(test_bow)
#### 

BOW Results:
Rank 0(1.0): Preliminary Report-International Algebraic Languag...
Rank 1(1.0): ALGOL Sub-Committee Report - Extensions...
Rank 2(1.0): The Use of Computers in Engineering Classroom Inst...
Rank 3(1.0): Report on a Conference of University Computing Cen...
Rank 4(1.0): Report on the Algorithmic Language ALGOL 60...


In [25]:
#### Please do not change this. This cell is used for grading.

In [26]:
#### Please do not change this. This cell is used for grading.

In [27]:
#### Please do not change this. This cell is used for grading.


---

### Section 3.2: TF-IDF (15 points) <a class="anchor" id="tfidf"></a>

Before we implement the tf-idf scoring functions, let's first write a function to compute the document frequencies of all words.  

#### 3.2.1 Document frequency (5 points)
Compute the document frequencies of all tokens in the collection.  

In [28]:
# TODO: Implement this! (5 points)
def compute_df(documents):
    """
        Compute the document frequency of all terms in the vocabulary
        Input: A list of documents
        Output: A dictionary with {token: document frequency)
    """
    # YOUR CODE HERE
    merge_list = []
    for doc in documents:
        merge_list.extend(set(doc))
    dict_ = Counter(merge_list)
    
    return dict_ 


In [29]:
#### Compute df based on the two configs

# get the document frequencies of each document
df_1 = compute_df([d[1] for d in doc_repr_1])
df_2 = compute_df([d[1] for d in doc_repr_2])

def get_df(index_set):
    assert index_set in {1, 2}
    return {
        1: df_1,
        2: df_2
    }[index_set]
####

In [30]:
#### Function check

print(df_1['computer'])
print(df_2['computer'])
####

597
11


---
#### 3.2.2 TF-IDF search (10 points)
Next, implement a function that computes a tf-idf score given a query.      

In [31]:
# TODO: Implement this! (10 points)
def tfidf_search(query, index_set):
    """
        Perform a search over all documents with the given query using tf-idf. 
        Note #1: You have to use the `get_index` (and the `get_df`) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    df = get_df(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    nr_docs = len(doc_repr_1)
    doc_scores = {}
    for query_word in processed_query:
        idf = math.log(nr_docs/df[query_word])
        if query_word in index:
            for (doc_id, tf) in index[query_word]:
                if doc_id not in doc_scores:
                    doc_scores[doc_id] = tf * idf
                else:
                    doc_scores[doc_id] += (tf * idf)

        sorted_scores = [(k, v) for k, v in sorted(doc_scores.items(), key=lambda item:item[1], reverse=True)]
        return sorted_scores




test_tfidf = tfidf_search("What articles exist which deal with TSS (Time Sharing System), an operating system for IBM computers?", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf)

TFIDF Results:
Rank 0(1.3e+01): Logic and Programming Languages\nLogic has been lo...
Rank 1(1.3e+01): Consumer Difficulties With Computerized Transactio...
Rank 2(8.9): What is Proprietary In Mathematical Programming?-I...
Rank 3(8.9): A Statistical Model for Console Behavior in Multiu...
Rank 4(8.9): Arguments for a Moratorium on the Construction\nof...


In [32]:
#### Function check
test_tfidf = tfidf_search("report", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf)
####

TFIDF Results:
Rank 0(2.3e+01): An Information Algebra - Phase I Report-Language\n...
Rank 1(2.3e+01): Rejuvenating Experimental Computer Science\nThis r...
Rank 2(1.2e+01): ALGOL 60 Confidential\nThe ALGOL 60 Report,* when ...
Rank 3(7.8): Automatic Abstracting and Indexing Survey and Reco...
Rank 4(7.8): A String Language for Symbol Manipulation Based on...


In [33]:
#### Please do not change this. This cell is used for grading.

In [34]:
#### Please do not change this. This cell is used for grading.

In [35]:
#### Please do not change this. This cell is used for grading.

--- 

### Section 3.3: Query Likelihood Model (35 points) <a class="anchor" id="qlm"></a>

In this section, you will implement a simple query likelihood model. 


#### 3.3.1 Naive QL (15 points)

First, let us implement a naive version of a QL model, assuming a multinomial unigram language model (with a uniform prior over the documents). 



In [36]:
#### Document length for normalization

def doc_lengths(documents):
    doc_lengths = {doc_id:len(doc) for (doc_id, doc) in documents}
    return doc_lengths

doc_lengths_1 = doc_lengths(doc_repr_1)
doc_lengths_2 = doc_lengths(doc_repr_2)

def get_doc_lengths(index_set):
    assert index_set in {1, 2}
    return {
        1: doc_lengths_1,
        2: doc_lengths_2
    }[index_set]
####

In [37]:
# TODO: Implement this! (15 points)
def naive_ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a naive QL model. 
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    # YOUR CODE HERE
    sum_of_all_words = sum(doc_lengths.values())                    
    result_dict = {}            
    for token in processed_query:
        if token in index:
            dict_token = dict(index[token])
            for (doc_id,length) in doc_lengths.items():
                term_1 = 0
                if doc_id in dict_token.keys():
                    term_1 = dict_token[doc_id]/doc_lengths[doc_id]    
                if doc_id not in result_dict:
                    result_dict[doc_id] = term_1
                else:
                    result_dict[doc_id] *= term_1
                    
    
    ql_scores_list = [(key, score) for key, score in result_dict.items()]
    ql_scores_list.sort(key=lambda x: x[1], reverse = True)
    return ql_scores_list  

In [38]:
#### Function check
test_naiveql = naive_ql_search("report", index_set=1)[:5]
print(f"Naive QL Results:")
print_results(test_naiveql)
####

Naive QL Results:
Rank 0(0.2): A Report Writer For COBOL...
Rank 1(0.2): A CRT Report Generating System...
Rank 2(0.17): Preliminary Report-International Algebraic Languag...
Rank 3(0.17): Supplement to the ALGOL 60 Report...
Rank 4(0.14): ALGOL Sub-Committee Report - Extensions...


In [39]:
#### Please do not change this. This cell is used for grading.

In [40]:
#### Please do not change this. This cell is used for grading.

In [41]:
#### Please do not change this. This cell is used for grading.

In [42]:
#### Please do not change this. This cell is used for grading.

---
#### 3.3.2 QL (20 points)
Now, let's implement a QL model that handles the issues with the naive version. In particular, you will implement a QL model with Jelinek-Mercer Smoothing. That means an interpolated score is computed per word - one term is the same as the previous naive version, and the second term comes from a unigram language model. In addition, you should accumulate the scores by summing the **log** (smoothed) probability which leads to better numerical stability.

In [43]:
# TODO: Implement this! (20 points)

# YOUR CODE HERE
def ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a QL model 
        with Jelinek-Mercer Smoothing (set smoothing=0.1). 
        
        
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    smoothing = 0.9
    sum_of_all_words = sum(doc_lengths.values())
                    
    result_dict = {}            
    for token in processed_query:
        if token in index:
            sum_freq = 0
            [sum_freq := sum_freq + x[1] for x in index[token]]
            term_2= (1-smoothing) * (sum_freq / sum_of_all_words)
            dict_token = dict(index[token])
            for (doc_id,length) in doc_lengths.items():
                term_1 = 0
                if doc_id in dict_token.keys():
                    term_1 = smoothing * (dict_token[doc_id]/doc_lengths[doc_id])    
                if doc_id not in result_dict:
                    result_dict[doc_id] = math.log(term_1 + term_2)
                else:
                    result_dict[doc_id] += math.log(term_1 + term_2)
                    
    
    ql_scores_list = [(key, score) for key, score in result_dict.items()]
    ql_scores_list.sort(key=lambda x: x[1], reverse = True)
    return ql_scores_list  

In [44]:
#### Function check
test_ql_results = ql_search("report", index_set=1)[:5]
print_results(test_ql_results)
print()
test_ql_results_long = ql_search("report " * 10, index_set=1)[:5]
print_results(test_ql_results_long)
####

Rank 0(-1.7): A Report Writer For COBOL...
Rank 1(-1.7): A CRT Report Generating System...
Rank 2(-1.9): Preliminary Report-International Algebraic Languag...
Rank 3(-1.9): Supplement to the ALGOL 60 Report...
Rank 4(-2.1): ALGOL Sub-Committee Report - Extensions...

Rank 0(-1.7e+01): A Report Writer For COBOL...
Rank 1(-1.7e+01): A CRT Report Generating System...
Rank 2(-1.9e+01): Preliminary Report-International Algebraic Languag...
Rank 3(-1.9e+01): Supplement to the ALGOL 60 Report...
Rank 4(-2.1e+01): ALGOL Sub-Committee Report - Extensions...


In [45]:
#### Please do not change this. This cell is used for grading.

In [46]:
#### Please do not change this. This cell is used for grading.

In [47]:
#### Please do not change this. This cell is used for grading.

In [48]:
#### Please do not change this. This cell is used for grading.

--- 

### Section 3.4: BM25 (20 points) <a class="anchor" id="bm25"></a>

In this section, we will implement the BM25 scoring function. 


In [49]:
# TODO: Implement this! (20 points)
def bm25_search(query, index_set):
    """
        Perform a search over all documents with the given query using BM25. Use k_1 = 1.5 and b = 0.75
        Note #1: You have to use the `get_index` (and `get_doc_lengths`) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    df = get_df(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # YOUR CODE HERE
    b = 0.75
    k_1 = 1.5
    
    num_of_docs = len(doc_lengths)
    avg_doc_length = sum(doc_lengths[key] for key in doc_lengths) / num_of_docs
    
    bm25_scores = {}
    for token in processed_query:
        if token in index:
            token_freq = df[token]
            for doc_id, num_of_occurences in index[token]:
                doc_length = doc_lengths[doc_id]
                idf = math.log((num_of_docs - token_freq + 0.5) / (token_freq + 0.5) + 1)
                K = k_1 * (1 - b + b * (doc_length / avg_doc_length))
                score = idf * (num_of_occurences * (k_1 + 1)) / (num_of_occurences + K)
                if doc_id in bm25_scores:
                    bm25_scores[doc_id] += score
                else:
                    bm25_scores[doc_id] = score 

    bm25_scores_list = [(key, score) for key, score in bm25_scores.items()]
    bm25_scores_list.sort(key=lambda x: x[1], reverse = True)   
    
    return bm25_scores_list 
    

In [50]:
#### Function check
test_bm25_results = bm25_search("report", index_set=1)[:5]
print_results(test_bm25_results)
####

Rank 0(6.7): A Report Writer For COBOL...
Rank 1(6.7): A CRT Report Generating System...
Rank 2(6.6): Preliminary Report-International Algebraic Languag...
Rank 3(6.6): Supplement to the ALGOL 60 Report...
Rank 4(6.5): ALGOL Sub-Committee Report - Extensions...


In [51]:
#### Please do not change this. This cell is used for grading.

In [52]:
#### Please do not change this. This cell is used for grading.

In [53]:
#### Please do not change this. This cell is used for grading.

In [54]:
#### Please do not change this. This cell is used for grading.


---

### 3.5. Test Your Functions

The widget below allows you to play with the search functions you've written so far. Use this to test your search functions and ensure that they work as expected.

In [55]:
#### Highlighter function
# class for results
ResultRow = namedtuple("ResultRow", ["doc_id", "snippet", "score"])
# doc_id -> doc
docs_by_id = dict((d[0], d[1]) for d in docs)

def highlight_text(document, query, tol=17):
    import re
    tokens = tokenize(query)
    regex = "|".join(f"(\\b{t}\\b)" for t in tokens)
    regex = re.compile(regex, flags=re.IGNORECASE)
    output = ""
    i = 0
    for m in regex.finditer(document):
        start_idx = max(0, m.start() - tol)
        end_idx = min(len(document), m.end() + tol)
        output += "".join(["...",
                        document[start_idx:m.start()],
                        "<strong>",
                        document[m.start():m.end()],
                        "</strong>",
                        document[m.end():end_idx],
                        "..."])
    return output.replace("\n", " ")


def make_results(query, search_fn, index_set):
    results = []
    for doc_id, score in search_fn(query, index_set):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results
####

In [56]:
# TODO: Set this to the function you want to test!
# this function should take in a query (string)
# and return a sorted list of (doc_id, score) 
# with the most relevant document in the first position
search_fn = bm25_search
index_set = 1

text = widgets.Text(description="Search Bar", width=200)
display(text)

def handle_submit(sender):
    print(f"Searching for: '{sender.value}'")
    
    results = make_results(sender.value, search_fn, index_set)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit)

Text(value='', description='Search Bar')

---

## Section 4: Evaluation (40 points) <a class="anchor" id="evaluation"></a>

[Back to Part 1](#part1)

Before we jump in and implement an algorithm for retrieval, we first have to learn how to evaluate such a system. In particular, we will work with offline evaluation metrics. These metrics are computed on a dataset with known relevance judgements.

Implement the following evaluation metrics. 

1. Precision (7 points)
2. Recall (7 points)
3. Mean Average Precision (12 points)
4. Expected Reciprocal Rank (12 points)

---
### 4.1 Read relevance labels (2 points)

Let's take a look at the `qrels.text` file, which contains the ground truth relevance scores. The relevance labels for CACM are binary - either 0 or 1. 


In [57]:
!head ./datasets/qrels.text

01 1410  0 0
01 1572  0 0
01 1605  0 0
01 2020  0 0
01 2358  0 0
02 2434  0 0
02 2863  0 0
02 3078  0 0
03 1134  0 0
03 1613  0 0


---
**Implementation (2 points):**
The first column is the query_id and the second column is the document_id. You can safely ignore the 3rd and 4th columns. Write a function to read in the file:

In [58]:
# TODO: Implement this! (2 points)
def read_qrels(root_folder = "./datasets/"):
    """
        Reads the qrels.text file. 
        Output: A dictionary: query_id -> [list of relevant documents]
    """
    # YOUR CODE HERE
    qrels_dict = {}
    with open(root_folder + 'qrels.text') as f:
        for line in f:
            line_split = line.split(' ')
            q_id = str(int(line_split[0]))
            doc_id = line_split[1]
            if q_id in qrels_dict:
                qrels_dict[q_id].append(doc_id)
            else:
                qrels_dict[q_id] = [doc_id]
        
    return qrels_dict

In [59]:
#### Function check
qrels = read_qrels()

assert len(qrels) == 52, "There should be 52 queries with relevance judgements"
assert sum(len(j) for j in qrels.values()) == 796, "There should be a total of 796 Relevance Judgements"
####

---
**Note:** For a given query `query_id`, you can assume that documents *not* in `qrels[query_id]` are not relevant to `query_id`. 


---
### 4.2 Precision (7 points)
Implement the `precision@k` metric:

In [60]:
# TODO: Implement this! (7 points)
def precision_k(results, relevant_docs, k):
    """
        Compute Precision@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), 
                    with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Precision@K
    """
    # YOUR CODE HERE
    
    relevant_doc_set = set(relevant_docs)
    results_doc_set = {k for k, v in results[:k]}

    numerator = len(relevant_doc_set.intersection(results_doc_set))
    
    precision = numerator / k


    return precision

In [61]:
#### Function check
qid = queries[0][0]
qtext = queries[0][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
precision = precision_k(results, qrels[qid], 10)
print(f'precision@10 = {precision}')
####

query:
 What articles exist which deal with TSS (Time Sharing System), an
operating system for IBM computers?
precision@10 = 0.2


---
### 4.3 Recall (7 points)
Implement the `recall@k` metric:

In [62]:
# TODO: Implement this! (7 points)
def recall_k(results, relevant_docs, k):
    """
        Compute Recall@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Recall@K
    """
    # YOUR CODE HERE
    relevant_doc_set = set(relevant_docs)
    results_doc_set = {k for k, v in results[:k]}

    numerator = len(relevant_doc_set.intersection(results_doc_set))
    denominator = len(relevant_doc_set)
    
    recall = numerator / denominator if denominator > 0 else 0

    return recall


In [63]:
#### Function check
qid = queries[10][0]
qtext = queries[10][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
recall = recall_k(results, qrels[qid], 10)
print(f'recall@10 = {recall}')
####

query:
 SETL, Very High Level Languages
recall@10 = 0.3157894736842105


---
### 4.4 Mean Average Precision (12 points)
Implement the `map` metric:

In [64]:
# TODO: Implement this! (12 points)
def average_precision(results, relevant_docs):
    """
        Compute Average Precision (for a single query - the results are 
        averaged across queries to get MAP in the next few cells)
        Hint: You can use the recall_k and precision_k functions here!
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: Average Precision
    """
    # YOUR CODE HERE
    precisions = []
    relevant_results = []
    for i in range(len(results)):
        cur_precision = precision_k(results, relevant_docs, i+1)
        doc_id, _ = results[i]
        if doc_id in relevant_docs:
            precisions.append(cur_precision)

    return sum(precisions)/len(precisions) if len(precisions) > 0 else 0

    

    
        


In [65]:
#### Function check
qid = queries[20][0]
qtext = queries[20][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
mean_ap = average_precision(results, qrels[qid])
print(f'MAP = {mean_ap}')
####

query:
 computational complexity, intractability, class-complete reductions,
algorithms and efficiency
MAP = 0.13885003916324018


---
### 4.5 Expected Reciprocal Rank (12 points)
Implement the `err` metric:

In [66]:
# TODO: Implement this! (12 points)
def err(results, relevant_docs):
    """
        Compute the expected reciprocal rank.
        Hint: https://dl.acm.org/doi/pdf/10.1145/1645953.1646033?download=true
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: ERR
        
    """
    # YOUR CODE HERE
    # The relevance grade is binary, so when a document is irrelevant, we get (2^0-1)/2 = 0
    # For a relevant document, this is (2^1-1)/2 = 0.5, hence the R of 0 or 0.5.
    p, ERR = 1, 0
    nr_docs = len(results)
    for r in range(nr_docs):
        cur_doc = results[r][0]
        if cur_doc in relevant_docs:
            R = 0.5
        else:
            R = 0
        ERR += p * R/(r + 1)
        p *= (1-R)
    
    return ERR

In [67]:
#### Function check
qid = queries[30][0]
qtext = queries[30][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
ERR = err(results, qrels[qid])
print(f'ERR = {ERR}')
####

query:
 I'd like to find articles describing the use of singular value decomposition
in digital image processing.  Applications include finding approximations
to the original image and restoring images that are subject to noise. An
article on the subject is H.C. Andrews and C.L. Patterson "Outer product
expansions and their uses in digital image processing", American Mathematical
Monthly, vol. 82.
ERR = 0.3333333333333333


---
### 4.6 Evaluate Search Functions

Let's define some metrics@k using [partial functions](https://docs.python.org/3/library/functools.html#functools.partial)

In [107]:
#### metrics@k functions

recall_at_1 = partial(recall_k, k=1)
recall_at_5 = partial(recall_k, k=5)
recall_at_10 = partial(recall_k, k=10)
precision_at_1 = partial(precision_k, k=1)
precision_at_5 = partial(precision_k, k=5)
precision_at_10 = partial(precision_k, k=10)


list_of_metrics = [
    ("ERR", err),
    ("MAP", average_precision),
    ("Recall@1",recall_at_1),
    ("Recall@5", recall_at_5),
    ("Recall@10", recall_at_10),
    ("Precision@1", precision_at_1),
    ("Precision@5", precision_at_5),
    ("Precision@10", precision_at_10)]
####

---

The following function evaluates a `search_fn` using the `metric_fn`. Note that the final number is averaged over all the queries

In [110]:
#### Evaluate a search function

list_of_search_fns = [
    ("BOW", bow_search),
    ("TF-IDF", tfidf_search),
    ("NaiveQL", naive_ql_search),
    ("QL", ql_search),
    ("BM25", bm25_search)
]

def evaluate_search_fn(search_fn, metric_fns, index_set=None):
    # build a dict query_id -> query 
    queries_by_id = dict((q[0], q[1]) for q in queries)
    metrics = {}
    for metric, metric_fn in metric_fns:
        metrics[metric] = np.zeros(len(qrels), dtype=np.float32)
    
    for i, (query_id, relevant_docs) in enumerate(qrels.items()):
        query = queries_by_id[query_id]
        if index_set:
            results = search_fn(query, index_set)
        else:
            results = search_fn(query)
        
        for metric, metric_fn in metric_fns:
            metrics[metric][i] = metric_fn(results, relevant_docs)

    
    
    final_dict = {}
    for metric, metric_vals in metrics.items():
        final_dict[metric] = metric_vals.mean()
    
    return final_dict
####

## Section 5: Analysis (30 points) <a class="anchor" id="analysis"></a>

[Back to Part 1](#part1)

In the final section of Part1, we will compare the different term-based IR algorithms and different preprocessing configurations and analyze their advantages and disadvantages.

### Section 5.1: Plot (20 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

**Rubric:**
- Each Metric is plotted: 7 points
- Each Method is plotted: 7 points
- Clear titles, x label, y labels and legends (if applicable): 6 points

In [118]:
# YOUR CODE HERE
toplot = {'ERR':[[],[]],
          'MAP':[[],[]],
          'Recall@1':[[],[]],
          'Recall@5':[[],[]],
          'Recall@10':[[],[]],
          'Precision@1':[[],[]],
          'Precision@5':[[],[]],
          'Precision@10':[[],[]]
         }

for search_name, search_fn in list_of_search_fns:
    index_set_res_1 = evaluate_search_fn(search_fn,list_of_metrics,1)
    for key,value in index_set_res_1.items():
        toplot[key][0].append(value)
    index_set_res_2 = evaluate_search_fn(search_fn,list_of_metrics,2)
    for key,value in index_set_res_2.items():
        toplot[key][1].append(value)

n_groups = len(list_of_search_fns)        
for (name,values) in toplot.items():
        dict_keys = dict(list_of_search_fns).keys()
        score_1 = values[0]
        score_2 = values[1]
        # create plot
        fig, ax = plt.subplots()
        index = np.arange(n_groups)
        bar_width = 0.35
        opacity = 0.8

        rects1 = plt.bar(index, score_1, bar_width,
        alpha=opacity,
        color='b',
        label='Index set = 1')

        rects2 = plt.bar(index + bar_width, score_2, bar_width,
        alpha=opacity,
        color='g',
        label='Index set = 2')

        plt.xlabel('Search functions')
        plt.ylabel('Results')
        plt.title('Results using ' + name + ' metric')
        plt.xticks(index + bar_width, dict_keys )
        plt.legend()

        plt.tight_layout()
        plt.xticks(rotation=45)
        plt.show()

KeyboardInterrupt: 

---
### Section 5.2: Summary (10 points)
Write a summary of what you observe in the results.
Your summary should compare results across the 2 indices and the methods being used. State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations.      

- Highest value - Precision@1 - BM25 - more than 0.5
- BM25 has the highest value using each metric
- NaiveQl - almost everytime the lowest value - reason : 0 multiplication
- QL model is significantly improve the score - reason: using smoothing insted of just 0 multiplication - using different weights for each word
- scores increase with higher partial values using Recall
- scores decrease using Precision with higher partial functions
- TF-IDF - index set 1 is better - 8 of 7 cases
    - in another cases index 2 is works better (only Recall1 + BM25 is an exception)

---
---
# Part 2: Semantic-based Matching (85 points) <a class="anchor" id="part2"></a>

[Back to top](#top)

We will now experiment with methods that go beyond lexical methods like TF-IDF, which operate at the word level and are high dimensional and sparse, and look at methods which constructs low dimensional dense representations of queries and documents. 

Since these low-dimensional methods have a higher time complexity, they are typically used in conjunction with methods like BM-25. That is, instead of searching through potentially million documents to find matches using low dimensional vectors, a list of K documents are retrieved using BM25, and then **re-ranked** using the other method. This is the method that is going to be applied in the following exercises. 

LSI/LDA takes documents that are similar on a semantic level - for instance, if they are describing the same topic - and projects them into nearby vectors, despite having low lexical overlap.

In this assignment, you will use `gensim` to create LSI/LDA models and use them in re-ranking. 

**Note**: The following exercises only uses `doc_repr_2` and `config_2`

Table of contents:
- [Section 6: LSI](#lsi) (15 points)
- [Section 7: LDA](#lda) (10 points)
- [Section 8: Word2Vec/Doc2Vec](#2vec) (20 points)
- [Section 8: Re-ranking](#reranking) (10 points)
- [Section 9: Re-ranking Evaluation](#reranking_eval) (30 points)

---
## Section 6: Latent Semantic Indexing (LSI) (15 points) <a class="anchor" id="lsi"></a>

[Back to Part 2](#part2)

LSI is one of the methods to embed the queries and documents into vectors. It is based on a method similar to Principal Component Analysis (PCA) for obtaining a dense concept matrix out of the sparse term-document matrix.

See [wikipedia](https://en.wikipedia.org/wiki/Latent_semantic_analysis), particularly [#Mathematics_of_LSI](https://en.wikipedia.org/wiki/Latent_semantic_analysis#Mathematics_of_LSI).

In [71]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel, Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import downloader as g_downloader
# gensim uses logging, so set it up 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

---
### Section 6.1: Cosine Similarity (5 points)<a class="anchor" id="cosing_sim"></a>
Before we begin, let us first define our method of similarity for the LSI model, the cosine similarity:

$$\text{similarity} = \cos(\theta) = {\mathbf{A} \cdot \mathbf{B} \over \|\mathbf{A}\| \|\mathbf{B}\|} = \frac{ \sum\limits_{i=1}^{n}{A_i  B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{A_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{B_i^2}} }$$

Since we are using gensim, the types of vectors returned by their classes are of the form defined below (they are not just simple vectors):

In [72]:
# 1, 2, 3 are either latent dimensions (LSI), or topics (LDA)
# The second value in each tuple is a number (LSI) or a probability (LDA)  
example_vec_1 = [(1, 0.2), (2, 0.3), (3, 0.4)]
example_vec_2 = [(1, 0.2), (2, 0.7), (3, 0.4)]

---
**Implementation (2+3 points):**
Now, implement the `dot product` operation on these types of vectors and using this operator, implement the `cosine similarity` (don't forget: two functions to implement!):

In [73]:
# TODO: Implement this! (2 points)
def dot(vec_1,vec_2): 
    """
        vec_1 and vec_2 are of the form: [(int, float), (int, float), ...]
        Return the dot product of two such vectors, computed only on the floats
        You can assume that the lengths of the vectors are the same, and the dimensions are aligned 
            i.e you won't get: vec_1 = [(1, 0.2)] ; vec_2 = [(2, 0.3)] 
                                (dimensions are unaligned and lengths are different)
    """
    # YOUR CODE HERE
    dot_product = 0
    for i in range(len(vec_1)):
        dot_product +=  vec_1[i][1] * vec_2[i][1]
    
    return dot_product

# TODO: Implement this! (3 points)
def cosine_sim(vec_1, vec_2):
    # YOUR CODE HERE
    similarity = 0
    res_a = 0
    res_b = 0
    for i in range(len(vec_1)):
        res_a += vec_1[i][1] ** 2
        res_b += vec_2[i][1] ** 2
    numerator = dot(vec_1,vec_2)
    
    # When there exists a zero vector
    # This happens when none of the tokens in the query are in the w2v model.
    # As per instruction, we return a similarity score of 0.
    if res_a == 0 or res_b == 0:
        return 0
    else:
        denominator = math.sqrt(res_a)*math.sqrt(res_b)
        return numerator/denominator
    
    
print(cosine_sim(example_vec_1,example_vec_2))

0.9165587597202866


In [74]:
##### Function check
print(f'vectors: {(example_vec_1,example_vec_2)}')
print(f'dot product = {dot(example_vec_1,example_vec_2)}')
print(f'cosine similarity = {cosine_sim(example_vec_1,example_vec_2)}')
##### 

vectors: ([(1, 0.2), (2, 0.3), (3, 0.4)], [(1, 0.2), (2, 0.7), (3, 0.4)])
dot product = 0.41000000000000003
cosine similarity = 0.9165587597202866


In [75]:
#### Please do not change this. This cell is used for grading.

---
### Section 6.2: LSI Retrieval (10 points)<a class="anchor" id="lsi_retrieval"></a>
LSI retrieval is simply ranking the documents based on their cosine similarity to the query vector.
First, let's write a parent class for vector-based retrieval models:

In [76]:
class VectorSpaceRetrievalModel:
    """
        Parent class for Dense Vector Retrieval models
    """
    def __init__(self, doc_repr):
        """
            document_collection: 
                [
                    (doc_id_1, [token 1, token 2, ...]), 
                    (doc_id_2, [token 1, token 2, ....]) 
                    ...
                ]

        """
        self.doc_repr = doc_repr
        self.documents = [_[1] for _ in self.doc_repr]
        
        # construct a dictionary
        self.dictionary = Dictionary(self.documents)
        # Filter out words that occur less than 20 documents, or more than 50% of the documents.
        self.dictionary.filter_extremes(no_below=10)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.documents]
    
        # Make a index to word dictionary.
        temp = self.dictionary[0]  # This is only to "load" the dictionary.
        self.id2word = self.dictionary.id2token
        
        # this is set by the train_model function
        self.model = None
        
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        vectors = {}
        for (doc_id, _), cc in zip(self.doc_repr, self.corpus):
            vectors[doc_id] = self.model[cc]
        return vectors

    def vectorize_query(self, query):
        # Note the use of config_2 here!
        query = process_text(query, **config_2)
        query_vector = self.dictionary.doc2bow(query)
        return self.model[query_vector]
    
    def train_model(self):
        """
            Trains a model and sets the 'self.model' variable. 
            Make sure to use the variables created in the __init__ method.
            e.g the variables which may be useful: {corpus, dictionary, id2word}
        """
        raise NotImplementedError()

---
**Implementation (5 points):**
Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. Normally, the hyperaparameter space will be searched using grid search / other methods - in this assignment we have provided the hyperparameters for you.

The last two lines of code train an LSI model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [77]:
# TODO: Implement this! (5 points)
class LsiRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.num_topics = 100
        self.chunksize = 2000
    
    def train_model(self):
        # YOUR CODE HERE
        self.model = LsiModel(self.corpus, id2word = self.id2word, num_topics = self.num_topics, chunksize = self.chunksize)

In [78]:
##### Function check
lsi = LsiRetrievalModel(doc_repr_2)
lsi.train_model()

# you can now get an LSI vector for a given query in the following way:
lsi.vectorize_query("report")
##### 

2021-02-18 10:30:25,445 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 10:30:25,599 : INFO : built Dictionary(6204 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 196845 corpus positions)
2021-02-18 10:30:25,605 : INFO : discarding 4795 tokens: [('of', 2061), ('repeat', 8), ('glossari', 7), ('inspect', 8), ('the', 1794), ('uncol', 2), ('a', 1807), ('rung', 9), ('secant', 2), ('.', 1603)]...
2021-02-18 10:30:25,606 : INFO : keeping 1409 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 10:30:25,608 : INFO : resulting dictionary: Dictionary(1409 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 10:30:25,693 : INFO : using serial LSI version on this node
2021-02-18 10:30:25,693 : INFO : updating model with new documents
2021-02-18 10:30:25,694 : INFO : preparing a new chunk of documents
2021-02-18 10:30:25,707 : INFO : using 100 extra sample

[(0, 0.009993383376755333),
 (1, -0.017277176070156452),
 (2, -0.0070684462210846115),
 (3, -0.0019305810002882575),
 (4, -0.006814029157683152),
 (5, 0.001447915351095468),
 (6, 0.005123062741064224),
 (7, -0.027669981065930667),
 (8, -0.007060240619858028),
 (9, -0.011944077358479013),
 (10, 0.018472384393366865),
 (11, 0.014778926332597975),
 (12, 0.041351660623568084),
 (13, -0.005843028236306556),
 (14, -0.0002639276717660089),
 (15, 0.001141290606981843),
 (16, 0.014582450503214705),
 (17, -0.0012194012779183118),
 (18, 0.016239260034582884),
 (19, -0.01638265224570167),
 (20, 0.021465739665252448),
 (21, -0.007911975204506719),
 (22, 0.007245535246986758),
 (23, 0.01882757438638228),
 (24, -0.039342422956939814),
 (25, -0.03790560394169666),
 (26, -0.005868167898832569),
 (27, 0.015157818608140658),
 (28, 0.010417381741452736),
 (29, -0.016672511107195916),
 (30, -0.0010495301914918423),
 (31, -0.0012180417246531426),
 (32, -0.009900954932706998),
 (33, -0.011971842357891694),
 

\#### Please do not change this. This cell is used for grading.

---
Next, implement a basic ranking class for vector space retrieval (used for all semantic methods): 

In [79]:
# TODO: Implement this! (5 points)
class DenseRetrievalRanker:
    def __init__(self, vsrm, similarity_fn):
        """
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.vsrm = vsrm 
        self.vectorized_documents = self.vsrm.vectorize_documents()
        self.similarity_fn = similarity_fn
    
    def _compute_sim(self, query_vector):
        """
            Compute the similarity of `query_vector` to documents in 
            `self.vectorized_documents` using `self.similarity_fn`
            Returns a list of (doc_id, score) tuples
        """
        # YOUR CODE HERE
        result_list = []
        self.query_vector = query_vector
        for key, doc in self.vectorized_documents.items():
            if len(doc) != 0:
                score = self.similarity_fn(doc,self.query_vector)
                result_list.append((key,score))               
        return result_list
    
    def search(self, query):
        scores = self._compute_sim(self.vsrm.vectorize_query(query))
        scores.sort(key=lambda _:-_[1])
        return scores 

In [80]:
##### Function check
drm_lsi = DenseRetrievalRanker(lsi, cosine_sim)
drm_lsi.search("report")[:5]
##### 

[('2531', 0.4878682192325991),
 ('53', 0.4484562401900328),
 ('947', 0.4463261988701967),
 ('3160', 0.4358787406613438),
 ('722', 0.3976848836849527)]

\#### Please do not change this. This cell is used for grading.

---
Now, you can test your LSI model in the following cell: try finding queries which are lexically different to documents, but semantically similar - does LSI work well for these queries?!

In [81]:
# test your LSI model
search_fn = drm_lsi.search

text = widgets.Text(description="Search Bar", width=200)
display(text)

def make_results_2(query, search_fn):
    results = []
    for doc_id, score in search_fn(query):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results

def handle_submit_2(sender):
    print(f"Searching for: '{sender.value}' (SEARCH FN: {search_fn})")
    
    results = make_results_2(sender.value, search_fn)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

---
## Section 7: Latent Dirichlet Allocation (LDA) (10 points) <a class="anchor" id="lda"></a>

[Back to Part 2](#part2)

The specifics of LDA is out of the scope of this assignment, but we will use the `gensim` implementation to perform search using LDA over our small document collection. The key thing to remember is that LDA, unlike LSI, outputs a topic **distribution**, not a vector. With that in mind, let's first define a similarity measure.


---
### Section 7.1: Jenson-Shannon divergence (5 points) <a class="anchor" id="js_sim"></a>

The Jenson-Shannon divergence is a symmetric and finite measure on two probability distributions (unlike the KL, which is neither). For identical distributions, the JSD is equal to 0, and since our code uses 0 as irrelevant and higher scores as relevant, we use `(1 - JSD)` as the score or 'similarity' in our setup

**Note**: the JSD is bounded to \[0,1\] only if we use log base 2. So please ensure that you're using `np.log2` instead of `np.log`

In [82]:
## TODO: Implement this! (5 points)
def jenson_shannon_divergence(vec_1, vec_2, assert_prob=False):
    """
        Computes the Jensen-Shannon divergence between two probability distributions. 
        NOTE: DO NOT RETURN 1 - JSD here, that is handled by the next function which is already implemented! 
        The inputs are *gensim* vectors - same as the vectors for the cosine_sim function
        assert_prob is a flag that checks if the inputs are proper probability distributions 
            i.e they sum to 1 and are positive - use this to check your inputs if needed. 
                (This is optional to implement, but recommended - 
                you can the default to False to save a few ms off the runtime)
    """
    # YOUR CODE HERE
    if assert_prob:
        sum_vec_1 = sum(value[1] for value in vec_1)
        sum_vec_2 = sum(value[1] for value in vec_2)
        if sum_vec_1 != 1:
            print('Input vec_1 is not in the correct format!')
            return
        if sum_vec_2 != 1:
            print('Input vec_2 is not in the correct format!')
            return
            
    vec_1 = np.array([value for id,value in vec_1])
    vec_2 = np.array([value for id,value in vec_2])
    
    d1 = vec_1*np.log2(vec_1/((vec_1+vec_2)/2))
    d2 = vec_2*np.log2(vec_2/((vec_1+vec_2)/2))
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*np.sum(d1+d2)
    
    return d
    
def jenson_shannon_sim(vec_1, vec_2, assert_prob=False):
    return 1 - jenson_shannon_divergence(vec_1, vec_2, assert_prob)


In [83]:
##### Function check
vec_1 = [(1, 0.3), (2, 0.4), (3, 0.3)]
vec_2 = [(1, 0.1), (2, 0.7), (3, 0.2)]
jenson_shannon_sim(vec_1, vec_2, assert_prob=True)
##### 

0.9251064410358459

---
### Section 7.2: LDA retrieval (5 points) <a class="anchor" id="lda_ret"></a>

Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. You do not need to set this. Normally, the hyperaparameter space will be searched using grid search / other methods. Note that training the LDA model might take some time

The last two lines of code train an LDA model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [84]:
# TODO: Implement this! (5 points)
class LdaRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # use these parameters in the train_model method
        self.num_topics = 100
        self.chunksize = 2000
        self.passes = 20
        self.iterations = 400
        self.eval_every = 10
        # this is need to get full vectors
        self.minimum_probability=0.0
        self.alpha='auto'
        self.eta='auto'
    
    
    def train_model(self):
        # YOUR CODE HERE
        self.model = LdaModel(self.corpus, num_topics = self.num_topics, chunksize = self.chunksize, id2word = self.id2word,
                              passes = self.passes, iterations = self.iterations, eval_every = self.eval_every, 
                              minimum_probability = self.minimum_probability, alpha = self.alpha, eta = self.eta)

In [85]:
##### Function check
lda = LdaRetrievalModel(doc_repr_2)
lda.train_model()

# you can now get an LDA vector for a given query in the following way:
lda.vectorize_query("report")
##### 

2021-02-18 10:30:26,946 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 10:30:27,103 : INFO : built Dictionary(6204 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 196845 corpus positions)
2021-02-18 10:30:27,109 : INFO : discarding 4795 tokens: [('of', 2061), ('repeat', 8), ('glossari', 7), ('inspect', 8), ('the', 1794), ('uncol', 2), ('a', 1807), ('rung', 9), ('secant', 2), ('.', 1603)]...
2021-02-18 10:30:27,109 : INFO : keeping 1409 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 10:30:27,113 : INFO : resulting dictionary: Dictionary(1409 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 10:30:27,197 : INFO : using autotuned alpha, starting with [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01

2021-02-18 10:30:35,784 : INFO : merging changes from 2000 documents into a model of 3204 documents
2021-02-18 10:30:35,805 : INFO : topic #6 (0.009): 0.001*"lead" + 0.001*"debug" + 0.001*"major" + 0.001*"divid" + 0.001*"column" + 0.001*"verifi" + 0.001*"repetit" + 0.001*"reduc" + 0.001*"quit" + 0.001*"maximum"
2021-02-18 10:30:35,806 : INFO : topic #85 (0.010): 0.348*"algol" + 0.221*"60" + 0.073*"iter" + 0.054*"languag" + 0.046*"algorithm" + 0.042*"(" + 0.041*"editor" + 0.029*"with" + 0.020*"and" + 0.019*"["
2021-02-18 10:30:35,806 : INFO : topic #41 (0.011): 0.221*"(" + 0.216*")" + 0.208*"algorithm" + 0.032*"integr" + 0.030*"s" + 0.025*"'" + 0.021*"[" + 0.018*"polynomi" + 0.018*"rule" + 0.017*"root"
2021-02-18 10:30:35,807 : INFO : topic #28 (0.011): 0.045*"is" + 0.039*"," + 0.037*"to" + 0.027*"and" + 0.026*"in" + 0.018*"tree" + 0.018*"that" + 0.017*"method" + 0.016*"be" + 0.016*"are"
2021-02-18 10:30:35,807 : INFO : topic #15 (0.011): 0.055*"," + 0.037*"and" + 0.035*"to" + 0.025*"in

2021-02-18 10:30:45,294 : INFO : topic #22 (0.010): 0.039*"for" + 0.035*"usag" + 0.033*"gener" + 0.030*"implement" + 0.024*"comput" + 0.023*"protect" + 0.022*"-" + 0.021*"time" + 0.020*"with" + 0.019*"design"
2021-02-18 10:30:45,294 : INFO : topic #28 (0.012): 0.046*"is" + 0.042*"," + 0.042*"to" + 0.027*"tree" + 0.027*"in" + 0.025*"and" + 0.023*"that" + 0.018*"be" + 0.017*"search" + 0.017*"are"
2021-02-18 10:30:45,295 : INFO : topic #15 (0.012): 0.063*"," + 0.042*"and" + 0.039*"to" + 0.025*"in" + 0.020*"is" + 0.018*"for" + 0.017*"system" + 0.017*"-" + 0.015*"comput" + 0.015*"program"
2021-02-18 10:30:45,295 : INFO : topic #41 (0.013): 0.287*"(" + 0.283*")" + 0.224*"algorithm" + 0.019*"integr" + 0.013*"normal" + 0.013*"rule" + 0.013*"evalu" + 0.013*"polynomi" + 0.011*"[" + 0.011*"find"
2021-02-18 10:30:45,297 : INFO : topic diff=0.596241, rho=0.466151
2021-02-18 10:30:45,306 : INFO : PROGRESS: pass 3, at document #2000/3204
2021-02-18 10:30:46,949 : INFO : optimized alpha [0.009979096, 

2021-02-18 10:30:52,538 : INFO : topic #28 (0.013): 0.047*"is" + 0.043*"to" + 0.043*"," + 0.027*"in" + 0.026*"tree" + 0.024*"that" + 0.024*"and" + 0.020*"be" + 0.018*"search" + 0.017*"are"
2021-02-18 10:30:52,539 : INFO : topic #41 (0.016): 0.323*")" + 0.321*"(" + 0.223*"algorithm" + 0.016*"integr" + 0.013*"evalu" + 0.012*"polynomi" + 0.011*"normal" + 0.007*"find" + 0.007*"x" + 0.007*"rule"
2021-02-18 10:30:52,540 : INFO : topic diff=0.643906, rho=0.389191
2021-02-18 10:30:54,536 : INFO : -6.486 per-word bound, 89.7 perplexity estimate based on a held-out corpus of 1204 documents with 79462 words
2021-02-18 10:30:54,536 : INFO : PROGRESS: pass 4, at document #3204/3204
2021-02-18 10:30:56,334 : INFO : optimized alpha [0.010250615, 0.010888591, 0.010650013, 0.010892246, 0.010855998, 0.010726702, 0.008344977, 0.011287438, 0.009989567, 0.010279896, 0.009923904, 0.01062096, 0.011218521, 0.0113455765, 0.0099697765, 0.013236896, 0.009648597, 0.0097051775, 0.009777251, 0.010724663, 0.01077251

2021-02-18 10:31:01,896 : INFO : topic #41 (0.020): 0.340*")" + 0.340*"(" + 0.224*"algorithm" + 0.009*"evalu" + 0.009*"normal" + 0.009*"polynomi" + 0.007*"x" + 0.007*"integr" + 0.006*"find" + 0.005*"rule"
2021-02-18 10:31:01,897 : INFO : topic diff=0.641777, rho=0.362690
2021-02-18 10:31:01,908 : INFO : PROGRESS: pass 6, at document #2000/3204
2021-02-18 10:31:03,563 : INFO : optimized alpha [0.0105057275, 0.011115565, 0.011040611, 0.011254378, 0.011112261, 0.010846811, 0.008088033, 0.011559493, 0.0101539325, 0.010487058, 0.009999992, 0.011013203, 0.011423195, 0.011918878, 0.010111745, 0.013676391, 0.0097727245, 0.009774637, 0.009808067, 0.011097525, 0.011110361, 0.011149469, 0.009797928, 0.011296285, 0.010676807, 0.009973418, 0.012101702, 0.010302287, 0.0141298855, 0.01047722, 0.009826882, 0.011437471, 0.011520546, 0.010302707, 0.010188347, 0.010253583, 0.00983145, 0.009993233, 0.010295239, 0.011081236, 0.010096203, 0.021744, 0.010680775, 0.010613715, 0.011735139, 0.010642618, 0.01048

...

2021-02-18 10:31:45,524 : INFO : topic #41 (0.049): 0.375*"(" + 0.371*")" + 0.204*"algorithm" + 0.007*"polynomi" + 0.007*"x" + 0.006*"evalu" + 0.006*"normal" + 0.003*"by" + 0.003*"type" + 0.002*"rule"
2021-02-18 10:31:45,525 : INFO : topic diff=0.239817, rho=0.253169
2021-02-18 10:31:45,536 : INFO : PROGRESS: pass 14, at document #2000/3204
2021-02-18 10:31:47,190 : INFO : optimized alpha [0.011989803, 0.012321061, 0.013212356, 0.0138192605, 0.013217388, 0.012106276, 0.0071477997, 0.013562157, 0.011585022, 0.011526246, 0.0107624335, 0.012851814, 0.013151601, 0.015835468, 0.011407357, 0.016347978, 0.010575809, 0.010504545, 0.010257074, 0.013196787, 0.013341441, 0.013055915, 0.011594778, 0.013891451, 0.012300947, 0.010731907, 0.014651703, 0.0112992525, 0.018288666, 0.011867358, 0.010819632, 0.013919396, 0.01555837, 0.011631058, 0.010972477, 0.010823496, 0.010423295, 0.010749688, 0.011124015, 0.014765464, 0.011194411, 0.05162719, 0.011719294, 0.011536433, 0.013936003, 0.012189343, 0.01214

2021-02-18 10:31:54,927 : INFO : PROGRESS: pass 15, at document #3204/3204
2021-02-18 10:31:56,853 : INFO : optimized alpha [0.012295881, 0.012581833, 0.013607819, 0.014357264, 0.013696128, 0.012420752, 0.007019373, 0.01400238, 0.011901131, 0.011703881, 0.010952432, 0.013156669, 0.013555751, 0.016763609, 0.011749984, 0.016914142, 0.010726368, 0.010660907, 0.010370777, 0.013557476, 0.013804919, 0.013438061, 0.012087794, 0.014335072, 0.012694386, 0.01091166, 0.015165497, 0.01150144, 0.019170454, 0.012251963, 0.011006571, 0.014403073, 0.016399281, 0.011946186, 0.011179558, 0.010962368, 0.010575177, 0.010916226, 0.011340798, 0.0157402, 0.011398469, 0.057350673, 0.011973495, 0.011740941, 0.014386916, 0.012419018, 0.012433415, 0.011015148, 0.010574137, 0.011794409, 0.011203414, 0.011366619, 0.013977286, 0.0139205, 0.012821552, 0.012530832, 0.013179058, 0.011499612, 0.01521093, 0.011763396, 0.019134218, 0.011608466, 0.010294926, 0.012763279, 0.010604634, 0.010729654, 0.011847005, 0.012176344,

2021-02-18 10:32:03,213 : INFO : merging changes from 2000 documents into a model of 3204 documents
2021-02-18 10:32:03,228 : INFO : topic #6 (0.007): 0.001*"lead" + 0.001*"debug" + 0.001*"major" + 0.001*"divid" + 0.001*"column" + 0.001*"verifi" + 0.001*"repetit" + 0.001*"reduc" + 0.001*"quit" + 0.001*"maximum"
2021-02-18 10:32:03,230 : INFO : topic #62 (0.010): 0.076*"precis" + 0.048*"divis" + 0.045*"is" + 0.043*"-" + 0.033*"to" + 0.028*"stage" + 0.027*"," + 0.026*"techniqu" + 0.025*"and" + 0.023*"thi"
2021-02-18 10:32:03,231 : INFO : topic #28 (0.020): 0.051*"to" + 0.046*"," + 0.044*"is" + 0.032*"tree" + 0.031*"in" + 0.030*"that" + 0.027*"be" + 0.019*"and" + 0.019*"search" + 0.018*"are"
2021-02-18 10:32:03,232 : INFO : topic #60 (0.020): 0.374*"s" + 0.349*"'" + 0.058*"$" + 0.055*"paramet" + 0.051*"))" + 0.048*"algorithm" + 0.016*"and" + 0.013*"for" + 0.009*"associ" + 0.006*"("
2021-02-18 10:32:03,233 : INFO : topic #41 (0.063): 0.379*"(" + 0.374*")" + 0.203*"algorithm" + 0.007*"polyn

2021-02-18 10:32:11,906 : INFO : topic #62 (0.010): 0.078*"precis" + 0.046*"divis" + 0.046*"is" + 0.045*"-" + 0.033*"stage" + 0.032*"to" + 0.028*"techniqu" + 0.026*"," + 0.025*"and" + 0.023*"thi"
2021-02-18 10:32:11,907 : INFO : topic #28 (0.020): 0.051*"to" + 0.046*"," + 0.043*"is" + 0.033*"tree" + 0.031*"in" + 0.030*"that" + 0.027*"be" + 0.019*"and" + 0.019*"search" + 0.018*"are"
2021-02-18 10:32:11,907 : INFO : topic #60 (0.021): 0.373*"s" + 0.348*"'" + 0.063*"$" + 0.056*"paramet" + 0.055*"))" + 0.048*"algorithm" + 0.014*"and" + 0.012*"for" + 0.008*"associ" + 0.004*"("
2021-02-18 10:32:11,908 : INFO : topic #41 (0.069): 0.384*"(" + 0.373*")" + 0.203*"algorithm" + 0.007*"x" + 0.006*"polynomi" + 0.006*"evalu" + 0.005*"normal" + 0.003*"type" + 0.002*"by" + 0.001*"bessel"
2021-02-18 10:32:11,909 : INFO : topic diff=0.162207, rho=0.220316
2021-02-18 10:32:11,921 : INFO : PROGRESS: pass 19, at document #2000/3204
2021-02-18 10:32:13,547 : INFO : optimized alpha [0.012942356, 0.012996811, 

[(0, 0.0053872056),
 (1, 0.0054048477),
 (2, 0.006024882),
 (3, 0.0063846884),
 (4, 0.0060265977),
 (5, 0.0053596455),
 (6, 0.0027794288),
 (7, 0.0061595314),
 (8, 0.005194039),
 (9, 0.0050039846),
 (10, 0.004702883),
 (11, 0.0057631168),
 (12, 0.005877402),
 (13, 0.007880025),
 (14, 0.0051609823),
 (15, 0.007471226),
 (16, 0.004640061),
 (17, 0.0045298347),
 (18, 0.004367164),
 (19, 0.005945957),
 (20, 0.006088355),
 (21, 0.005903203),
 (22, 0.005537191),
 (23, 0.0064112376),
 (24, 0.005672648),
 (25, 0.0046693976),
 (26, 0.006785768),
 (27, 0.0049426407),
 (28, 0.008606288),
 (29, 0.005364772),
 (30, 0.0047596404),
 (31, 0.006419124),
 (32, 0.0075984434),
 (33, 0.005216975),
 (34, 0.004788354),
 (35, 0.0046475213),
 (36, 0.004486063),
 (37, 0.0046707476),
 (38, 0.0048896596),
 (39, 0.007499246),
 (40, 0.0049687154),
 (41, 0.029717097),
 (42, 0.005119577),
 (43, 0.005027283),
 (44, 0.006324859),
 (45, 0.0054060235),
 (46, 0.005443865),
 (47, 0.004688085),
 (48, 0.004464348),
 (49, 0.0

\#### Please do not change this. This cell is used for grading.

---
Now we can use the `DenseRetrievalModel` class to obtain an LDA search function.
You can test your LDA model in the following cell: Try finding queries which are lexically different to documents, but semantically similar - does LDA work well for these queries?!

In [86]:
drm_lda = DenseRetrievalRanker(lda, jenson_shannon_sim)

# test your LDA model
search_fn = drm_lda.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

## Section 8: Word2Vec/Doc2Vec (20 points) <a class="anchor" id="2vec"></a>

[Back to Part 2](#part2)

We will implement two other methods here, the Word2Vec model and the Doc2Vec model, also using `gensim`. Word2Vec creates representations of words, not documents, so the word level vectors need to be aggregated to obtain a representation for the document. Here, we will simply take the mean of the vectors. 


A drawback of these models is that they need a lot of training data. Our dataset is tiny, so in addition to using a model trained on the data, we will also use a pre-trained model for Word2Vec (this will be automatically downloaded).     

*Note*:
1. The code in vectorize_documents / vectorize_query should return gensim-like vectors i.e `[(dim, val), .. (dim, val)]`. 
2. For Word2Vec: You should also handle the following two cases: (a) A word in the query is not present in the vocabulary of the model and (b) none of the words in the query are present in the model - you can return 0 scores for all documents in this case. For either of these, you can check if a `word` is present in the vocab by using `word in self.model`


In [87]:
# TODO: Implement this! (10 points)
class W2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # the dimensionality of the vectors
        self.size = 100 
        self.min_count = 1
    
    def train_model(self):
        """
        Trains the W2V model
        """
        # YOUR CODE HERE
        self.model = Word2Vec(self.documents, size=self.size, min_count=self.min_count) 
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        # YOUR CODE HERE
        vectors = {}
        for (doc_id, doc) in self.doc_repr:
            accum_vecs = []
            for token in doc:
                if token in self.model:
                    accum_vecs.append(self.model[token])

            aggregated_vec = np.mean(accum_vecs, axis=0)
            if not accum_vecs:
                vectors[doc_id] = list(zip(range(self.size), self.size * [0]))
            else:
                vectors[doc_id] = list(zip(range(self.size), aggregated_vec))
        return vectors

    def vectorize_query(self, query):
        """
        Vectorizes the query using the W2V model
        """
        query = process_text(query, **config_2)
        # YOUR CODE HERE
        accum_vecs = []
        for token in query:
            if token in self.model:
                accum_vecs.append(self.model[token])
        aggregated_vec = np.mean(accum_vecs, axis=0)
    
        # If no tokens are in the model
        if not accum_vecs:
            return list(zip(range(self.size), self.size * [0]))
        else:
            return list(zip(range(self.size), aggregated_vec))
    
    
class W2VPretrainedRetrievalModel(W2VRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        self.model_name = "word2vec-google-news-300"
        self.size = 300
    
    def train_model(self):
        """
        Loads the pretrained model
        """
        self.model = g_downloader.load(self.model_name)

w2v = W2VRetrievalModel(doc_repr_2)
w2v.train_model()

# you can now get a W2V vector for a given query in the following way:
w2v.vectorize_query("report")

2021-02-18 10:32:20,956 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 10:32:21,135 : INFO : built Dictionary(6204 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 196845 corpus positions)
2021-02-18 10:32:21,142 : INFO : discarding 4795 tokens: [('of', 2061), ('repeat', 8), ('glossari', 7), ('inspect', 8), ('the', 1794), ('uncol', 2), ('a', 1807), ('rung', 9), ('secant', 2), ('.', 1603)]...
2021-02-18 10:32:21,143 : INFO : keeping 1409 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 10:32:21,146 : INFO : resulting dictionary: Dictionary(1409 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 10:32:21,247 : INFO : collecting all words and their counts
2021-02-18 10:32:21,248 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-18 10:32:21,281 : INFO : collected 6204 word types from a corpus of 196845 raw word

[(0, -0.21797433),
 (1, 0.29363018),
 (2, 0.44798073),
 (3, 0.48633277),
 (4, -0.26308218),
 (5, 0.6163346),
 (6, 0.6194032),
 (7, -0.27473277),
 (8, 0.106308095),
 (9, 0.8754732),
 (10, 0.043273285),
 (11, 0.007342942),
 (12, 0.061213195),
 (13, 0.2148582),
 (14, 0.5103049),
 (15, -0.32976082),
 (16, 0.09507981),
 (17, 0.032684736),
 (18, -0.6046834),
 (19, -0.10366663),
 (20, 0.13782202),
 (21, -0.70971316),
 (22, -0.030011998),
 (23, 0.05774759),
 (24, 0.31920055),
 (25, -0.66972196),
 (26, 0.100932695),
 (27, -0.3690439),
 (28, 0.43471313),
 (29, 0.4112181),
 (30, 0.31732997),
 (31, 0.46848565),
 (32, -0.09791471),
 (33, 0.18068244),
 (34, -0.7082389),
 (35, -0.20685248),
 (36, -0.22399968),
 (37, -0.001143088),
 (38, 0.19116509),
 (39, 0.4104181),
 (40, 0.21436368),
 (41, 0.121224105),
 (42, -0.1883466),
 (43, -0.05598579),
 (44, 0.5078981),
 (45, -0.2831377),
 (46, 0.3485772),
 (47, 0.092254095),
 (48, -0.44009808),
 (49, -0.10001993),
 (50, 0.024670875),
 (51, -0.4263461),
 (52,

In [88]:
assert len(w2v.vectorize_query("report")) == 100
assert len(w2v.vectorize_query("this is a sentence that is not mellifluous")) == 100


<ipython-input-87-2f5bb1a63d99>:44: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-87-2f5bb1a63d99>:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  accum_vecs.append(self.model[token])


\#### Please do not change this. This cell is used for grading.

In [89]:
w2v_pretrained = W2VPretrainedRetrievalModel(doc_repr_2)
w2v_pretrained.train_model()

# you can now get an W2V vector for a given query in the following way:
w2v_pretrained.vectorize_query("report")

2021-02-18 10:32:23,036 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 10:32:23,279 : INFO : built Dictionary(6204 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 196845 corpus positions)
2021-02-18 10:32:23,290 : INFO : discarding 4795 tokens: [('of', 2061), ('repeat', 8), ('glossari', 7), ('inspect', 8), ('the', 1794), ('uncol', 2), ('a', 1807), ('rung', 9), ('secant', 2), ('.', 1603)]...
2021-02-18 10:32:23,291 : INFO : keeping 1409 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 10:32:23,295 : INFO : resulting dictionary: Dictionary(1409 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 10:32:23,866 : INFO : loading projection weights from C:\Users\Lenovo/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2021-02-18 10:33:02,816 : INFO : loaded (3000000, 300) matrix from C:\Users\Lenovo/gensim-data\word2vec-google-news

[(0, -0.14257812),
 (1, -0.1640625),
 (2, -0.09033203),
 (3, -0.11230469),
 (4, 0.100097656),
 (5, -0.041259766),
 (6, 0.048828125),
 (7, -0.13671875),
 (8, 0.19628906),
 (9, -0.13476562),
 (10, -0.017578125),
 (11, 0.032226562),
 (12, 0.095214844),
 (13, -0.10595703),
 (14, -0.16992188),
 (15, 0.041015625),
 (16, -0.26367188),
 (17, -0.0063171387),
 (18, -0.17773438),
 (19, -0.24023438),
 (20, 0.3515625),
 (21, -0.012207031),
 (22, -0.16210938),
 (23, -0.12060547),
 (24, 0.04321289),
 (25, 0.10986328),
 (26, 0.052490234),
 (27, 0.17871094),
 (28, -0.14550781),
 (29, 0.13769531),
 (30, -0.08203125),
 (31, -0.28320312),
 (32, -0.10888672),
 (33, -0.2890625),
 (34, 0.072265625),
 (35, -0.04736328),
 (36, 0.040283203),
 (37, 0.067871094),
 (38, 0.11669922),
 (39, 0.000831604),
 (40, 0.068359375),
 (41, 0.12011719),
 (42, -0.088378906),
 (43, 0.33789062),
 (44, -0.044677734),
 (45, -0.030151367),
 (46, 0.0076904297),
 (47, -0.021118164),
 (48, -0.25390625),
 (49, 0.14941406),
 (50, 0.39843

In [90]:
##### Function check

print(len(w2v_pretrained.vectorize_query("report")))
#####

300


In [91]:
drm_w2v = DenseRetrievalRanker(w2v, cosine_sim)

# test your LDA model
search_fn = drm_w2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

<ipython-input-87-2f5bb1a63d99>:26: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-87-2f5bb1a63d99>:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  accum_vecs.append(self.model[token])


Text(value='', description='Search Bar')

In [92]:
drm_w2v_pretrained = DenseRetrievalRanker(w2v_pretrained, cosine_sim)

# test your LDA model
search_fn = drm_w2v_pretrained.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

C:\Users\Lenovo\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Lenovo\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Text(value='', description='Search Bar')

**Implementation (10 points):**
For Doc2Vec, you will need to create a list of `TaggedDocument` instead of using the `self.corpus` or `self.documents` variable. Use the document id as the 'tag'.
  

In [93]:
# TODO: Implement this! (10 points)
class D2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.vector_size= 100
        self.min_count = 1
        self.epochs = 20
        
        # YOUR CODE HERE
        self.tagged_docs = [TaggedDocument(doc, [int(doc_id)]) for doc_id, doc in self.doc_repr]

    def train_model(self):
        # YOUR CODE HERE
        self.model = Doc2Vec(self.tagged_docs, vector_size=self.vector_size, min_count=self.min_count, epochs=self.epochs)
    
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        # YOUR CODE HERE
        vectors = {}
        for (doc_id, _) in self.doc_repr:
            vectors[doc_id] = list(zip(range(self.vector_size), self.model[int(doc_id)]))
        return vectors

    def vectorize_query(self, query): 
        query = process_text(query, **config_2)
        # YOUR CODE HERE
        # Here we just infer the vector for the query.
        return list(zip(range(self.vector_size), self.model.infer_vector(query)))
        
d2v = D2VRetrievalModel(doc_repr_2)
d2v.train_model()


# # you can now get an LSI vector for a given query in the following way:
d2v.vectorize_query("report")

2021-02-18 10:33:04,896 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-18 10:33:05,120 : INFO : built Dictionary(6204 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 196845 corpus positions)
2021-02-18 10:33:05,128 : INFO : discarding 4795 tokens: [('of', 2061), ('repeat', 8), ('glossari', 7), ('inspect', 8), ('the', 1794), ('uncol', 2), ('a', 1807), ('rung', 9), ('secant', 2), ('.', 1603)]...
2021-02-18 10:33:05,128 : INFO : keeping 1409 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-18 10:33:05,132 : INFO : resulting dictionary: Dictionary(1409 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-18 10:33:05,245 : INFO : collecting all words and their counts
2021-02-18 10:33:05,246 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-02-18 10:33:05,283 : INFO : collected 6204 word types and 3205 unique tags from a

2021-02-18 10:33:11,434 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-18 10:33:11,435 : INFO : EPOCH - 16 : training on 196845 raw words (140080 effective words) took 0.3s, 534943 effective words/s
2021-02-18 10:33:11,690 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-18 10:33:11,707 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-18 10:33:11,708 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-18 10:33:11,708 : INFO : EPOCH - 17 : training on 196845 raw words (139946 effective words) took 0.3s, 520439 effective words/s
2021-02-18 10:33:11,955 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-18 10:33:11,979 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-18 10:33:11,983 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-18 10:33:11,984 : INFO : EPOCH - 18 : training on 196845 raw words (139920 effectiv

[(0, -0.035835788),
 (1, 0.035669368),
 (2, 0.013635722),
 (3, 0.009951726),
 (4, -0.10988473),
 (5, 0.082064755),
 (6, 0.101332575),
 (7, -0.03577373),
 (8, 0.03522782),
 (9, 0.103660055),
 (10, 0.016160747),
 (11, -0.045502227),
 (12, -0.015544295),
 (13, 0.038512185),
 (14, 0.028655764),
 (15, 0.00067865336),
 (16, 0.01592007),
 (17, -0.028375827),
 (18, -0.06585895),
 (19, -0.0018624353),
 (20, 0.059122078),
 (21, -0.11374417),
 (22, -0.061846063),
 (23, 0.026505131),
 (24, 0.048032023),
 (25, -0.061747774),
 (26, 0.020185279),
 (27, -0.026829202),
 (28, 0.057540737),
 (29, 0.025608283),
 (30, -0.002159885),
 (31, 0.07973099),
 (32, -0.034286976),
 (33, -0.034795705),
 (34, -0.10407952),
 (35, 0.045472562),
 (36, -0.07186854),
 (37, 0.12587221),
 (38, -0.0064631733),
 (39, 0.019719083),
 (40, 0.061336774),
 (41, 0.002187198),
 (42, -0.052609995),
 (43, -0.055941883),
 (44, 0.10538186),
 (45, -0.06484728),
 (46, 0.054287948),
 (47, -0.0106153255),
 (48, 0.01953684),
 (49, -0.0182958

In [94]:
#### Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [95]:
drm_d2v = DenseRetrievalRanker(d2v, cosine_sim)

# test your LDA model
search_fn = drm_d2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

---
## Section 9: Re-ranking (10 points) <a class="anchor" id="reranking"></a>

[Back to Part 2](#part2)

To motivate the re-ranking perspective (i.e retrieve with lexical method + rerank with a semantic method), let's search using semantic methods and compare it to BM25's performance, along with their runtime:


In [96]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit drm_lsi.search(query)
print("LDA: ")
%timeit drm_lda.search(query)
print("W2V: ")
%timeit drm_w2v.search(query)
print("W2V(Pretrained): ")
%timeit drm_w2v_pretrained.search(query)
print("D2V:")
%timeit drm_d2v.search(query)

BM25: 
746 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
LSI: 
411 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
LDA: 
171 ms ± 9.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
W2V: 


<ipython-input-87-2f5bb1a63d99>:44: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-87-2f5bb1a63d99>:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  accum_vecs.append(self.model[token])


2.08 s ± 206 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
W2V(Pretrained): 
5.15 s ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
D2V:
1.75 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

**Implementation (10 points):**
Re-ranking involves retrieving a small set of documents using simple but fast methods like BM25 and then re-ranking them with the aid of semantic methods such as LDA or LSI. Implement the following class, which takes in an `initial_retrieval_fn` - the initial retrieval function and `vsrm` - an instance of the `VectorSpaceRetrievalModel` class (i.e LSI/LDA) as input. The search function should first retrieve an initial list of K documents, and then these documents are re-ranked using a semantic method. This not only makes retrieval faster, but semantic methods perform poorly when used in isolation, as you will find out.

In [238]:
# TODO: Implement this! (10 points)
class DenseRerankingModel:
    def __init__(self, initial_retrieval_fn, vsrm, similarity_fn):
        """
            initial_retrieval_fn: takes in a query and returns a list of [(doc_id, score)] (sorted)
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.ret = initial_retrieval_fn
        self.vsrm = vsrm
        self.similarity_fn = similarity_fn
        self.vectorized_documents = vsrm.vectorize_documents()
        
        assert len(self.vectorized_documents) == len(doc_repr_2)
    
    def search(self, query, K=50):
        """
            First, retrieve the top K results using the retrieval function
            Then, re-rank the results using the VSRM instance
        """
        # YOUR CODE HERE
        init_ranking = self.ret(query)[:K]
        doc_ids_init_ranking = [rank[0] for rank in init_ranking]
        rerank_docs = {}
        for doc_id in doc_ids_init_ranking:
            rerank_docs[doc_id] = self.vectorized_documents[doc_id]
        
        rerank_results = []
        query_vector = self.vsrm.vectorize_query(query)
        for key, doc_vector in rerank_docs.items():
            score = self.similarity_fn(query_vector, doc_vector)
            rerank_results.append((key,score))

        rerank_results.sort(key=lambda _:-_[1])
        return rerank_results

In [239]:
##### Function check
bm25_search_2 = partial(bm25_search, index_set=2)
lsi_rerank = DenseRerankingModel(bm25_search_2, lsi, cosine_sim)
lda_rerank = DenseRerankingModel(bm25_search_2, lda, jenson_shannon_sim)
w2v_rerank = DenseRerankingModel(bm25_search_2, w2v, cosine_sim)
w2v_pretrained_rerank = DenseRerankingModel(bm25_search_2, w2v_pretrained, cosine_sim)
d2v_rerank = DenseRerankingModel(bm25_search_2, d2v, cosine_sim)

##### 

<ipython-input-87-2f5bb1a63d99>:26: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-87-2f5bb1a63d99>:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  accum_vecs.append(self.model[token])


\#### Please do not change this. This cell is used for grading.

---
Now, let us time the new search functions:

In [240]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit lsi_rerank.search(query)
print("LDA: ")
%timeit lda_rerank.search(query)
print("W2V: ")
%timeit w2v_rerank.search(query)
print("W2V(Pretrained): ")
%timeit w2v_pretrained_rerank.search(query)
print("D2V:")
%timeit d2v_rerank.search(query)

BM25: 
713 µs ± 5.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
LSI: 
7.6 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
LDA: 
3.7 ms ± 78.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
W2V: 


<ipython-input-87-2f5bb1a63d99>:44: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-87-2f5bb1a63d99>:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  accum_vecs.append(self.model[token])


28.9 ms ± 677 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
W2V(Pretrained): 
83.6 ms ± 3.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
D2V:
34 ms ± 5.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


---
As you can see, it is much faster (but BM25 is still orders of magnitude faster).

---
## Section 10: Evaluation & Analysis (30 points) <a class="anchor" id="reranking_eval"></a>

[Back to Part 2](#part2)

[Previously](#evaluation) we have implemented some evaluation metrics and used them for measuring the ranking performance of term-based IR algorithms. In this section, we will do the same for semantic methods, both with and without re-ranking.

### Section 10.1: Plot (10 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

Then, gather only the re-ranking models, and plot and compare them with the results obtained in part 1 (only index set 2).

In [220]:
list_of_sem_search_fns = [
    ("lda", drm_lda.search),
    ("lsi", drm_lsi.search),
    ("w2v", drm_w2v.search),
    ("w2v_pretrained", drm_w2v_pretrained.search),
    ("d2v", drm_d2v.search),
    ("lsi_rr", lsi_rerank.search),
    ("lda_rr", lda_rerank.search),
    ("w2v_rr", w2v_rerank.search),
    ("w2v_pretrained_rr", w2v_pretrained_rerank.search),
    ("d2v_rr", d2v_rerank.search),
    
]

In [241]:
# YOUR CODE HERE
toplot = {'ERR':[],
          'MAP':[],
          'Recall@1':[],
          'Recall@5':[],
          'Recall@10':[],
          'Precision@1':[],
          'Precision@5':[],
          'Precision@10':[]
         }
toplot_rr = {'ERR':[],
          'MAP':[],
          'Recall@1':[],
          'Recall@5':[],
          'Recall@10':[],
          'Precision@1':[],
          'Precision@5':[],
          'Precision@10':[]
         }

for (name,fn) in list_of_sem_search_fns:
    if name[-2:] != 'rr':
        result = evaluate_search_fn(fn,list_of_metrics)
        for key,value in result.items():
            toplot[key].append(value)
    else:
        result = evaluate_search_fn(fn,list_of_metrics,2)
        for key,value in result.items():
            toplot_rr[key].append(value)
            
for (name,list_scores) in toplot.items():
    objects = dict(list_of_sem_search_fns[:5]).keys()
    y_pos = np.arange(len(objects))
    score = list_scores
    plt.bar(y_pos, score, align='center', alpha=0.5, color='b')
    plt.xticks(y_pos, objects)
    plt.ylabel('Scores')
    plt.title('Results using ' + name + ' metric')
    plt.show()
    
for (name,list_scores) in toplot_rr.items():
    objects = dict(list_of_sem_search_fns[5:]).keys()
    y_pos = np.arange(len(objects))
    score = list_scores
    plt.bar(y_pos, score, align='center', alpha=0.5, color='b')
    plt.xticks(y_pos, objects)
    plt.ylabel('Scores')
    plt.title('Results using ' + name + ' metric')
    plt.show()

KeyboardInterrupt: 

### Section 10.2: Summary (20 points)

Your summary should compare methods from Part 1 and Part 2 (only for index set 2). State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations. Consider the availability of data, scalability, domain/type of data, etc.

YOUR ANSWER HERE